# ChurnInsight — Pipeline Principal

Este notebook representa o **pipeline operacional canônico** do projeto **ChurnInsight**.  
Ele consolida, de forma **sequencial, reprodutível e auditável**, todo o fluxo de preparação de dados, definição da variável-alvo, treinamento e geração dos artefatos consumidos pela API de previsão.

Seu papel **não é exploratório**, mas **orquestrador**:  
todas as etapas aqui executadas refletem **decisões técnicas previamente definidas**, documentadas e versionadas nos materiais oficiais do projeto.

📘 **Documentação de referência obrigatória**  
- `pipeline_notebook.md` — descreve a **narrativa operacional** do pipeline, explicando *como* o notebook deve ser lido e interpretado.  
- `pipeline_elements.md` — documenta a **lógica técnica e semântica** de cada elemento exibido no notebook, explicitando contratos, decisões e artefatos produzidos.

Esses documentos complementam este notebook, fornecendo o contexto técnico e semântico das decisões apresentadas ao longo do pipeline.

---

## Estrutura do pipeline

O conteúdo é organizado em **etapas numeradas**, refletindo o fluxo real de transformação dos dados ao longo do projeto:

- Configuração do ambiente e contextualização do projeto  
- Ingestão e caracterização inicial do dataset  
- Diagnóstico de qualidade estrutural e tipagem  
- Conformidade ao contrato e preparação semântica dos dados  
- Tratamento de dados faltantes (com auditoria explícita)  
- Construção e validação da variável-alvo  
- Treinamento e avaliação do modelo  
- Geração dos artefatos finais para integração com a API  

Cada seção segue explicitamente o padrão:

**diagnóstico → decisão explícita → execução → auditoria**

Nenhuma transformação crítica ocorre de forma implícita ou silenciosa.

---

## Autoria e contribuições

Este material foi desenvolvido no contexto das atividades de **Data Science do projeto ChurnInsight**, com as seguintes contribuições:

- **Lilian Moraes** — Cientista de Dados  
- **Lucas Frigato** — Cientista de Dados  
- **Luedji Abayomi** — Cientista de Dados  
- **Fábio Aguiar** — Engenheiro de Dados  


## Contexto de Execução do Notebook
Esta célula estabelece o **contexto de execução** do notebook, tornando explícito o ponto de referência do projeto durante a leitura e execução das etapas seguintes.

Ela atua como um **contrato cognitivo**, ajudando o leitor a compreender de onde os módulos são resolvidos e como o notebook se posiciona dentro da estrutura do projeto, sem impor dependências funcionais ao pipeline.

In [2]:
from pathlib import Path
import sys

ROOT = Path.cwd()
while not (ROOT / "src").exists() and ROOT.parent != ROOT:
    ROOT = ROOT.parent

sys.path.insert(0, str(ROOT))

print("Project ROOT:", ROOT)

Project ROOT: C:\Users\fabio\Projetos DEV\Hackathon ONE\churninsight-hackathon-one


## 1 - Ingestão e diagnóstico inicial
> Esta seção estabelece o estado inicial do dataset após a ingestão, apresentando indicadores puramente diagnósticos de origem, dimensão, tipagem e presença de valores ausentes.
> Nenhuma transformação é aplicada nesta etapa.

In [3]:
from src.data.load_data import list_raw_files
list_raw_files()

,filename,extension,size_bytes,size_human
0,Bank Customer Churn Prediction.csv,csv,561600,548.44 KB
1,WA_Fn-UseC_-Telco-Customer-Churn.csv,csv,977501,954.59 KB


In [4]:
from src.data.load_data import load_and_report_raw_data
df = load_and_report_raw_data("Bank Customer Churn Prediction.csv")

## 2 - Qualidade Estrutural & Tipagem
> Esta seção avalia e consolida a qualidade estrutural do dataset, aplicando validações técnicas e conversões de tipo necessárias para garantir consistência operacional.
> As alterações são técnicas e não afetam a semântica de negócio.

In [5]:
from src.data.quality_typing import run_quality_and_typing_report
df = run_quality_and_typing_report(df)

Métrica,Antes,Depois,Δ
Linhas,10000.0,10000.0,0.0
Colunas,12.0,12.0,0.0
Memória (MB),1.8,1.8,0.0


## 3 - Conformidade ao Contrato (API) & Preparação Semântica dos Dados
> Esta seção estabelece o escopo semântico do pipeline, conectando o dataset validado ao contrato formal da API de previsão.
> Aqui são identificadas features, target e candidatos categóricos, sem aplicar transformações.

> Observação:  
> O target (`Churn`) já é fornecido pelo dataset no formato *Yes/No*.  
> Nesta seção, ele é apenas **identificado**, **isolado do escopo de diagnóstico** e **preservado estruturalmente**, sem qualquer transformação ou inferência adicional.

In [6]:
from src.data.contract_loader import load_contract_yaml
from src.features.contract_and_candidates import run_contract_and_candidates
from src.reporting.ui import render_contract_and_candidates_report

CONTRACT_PATH = "contracts/bank_churn.yaml"

cfg = load_contract_yaml(CONTRACT_PATH)
scope = cfg.to_scope()

payload_s31 = run_contract_and_candidates(df, scope=scope)
render_contract_and_candidates_report(payload_s31)
df = payload_s31["df"]

Métrica,Antes,Depois,Δ
Linhas,10000,10000,+0.00
Colunas,12,7,-5.00
Memória,1.80 MB,0.53 MB,-1.27 MB


column,dtype,n_unique,pct_unique,sample_values,reasons
active_member,int64,2,0.0002,"[1, 0]","numérico baixa cardinalidade, binário (yes/no)"
credit_card,int64,2,0.0002,"[1, 0]","numérico baixa cardinalidade, binário (yes/no)"
products_number,int64,4,0.0004,"[1, 3, 2, 4]",numérico baixa cardinalidade
tenure,int64,11,0.0011,"[2, 1, 8, 7, 4, 6]",numérico baixa cardinalidade
column,dtype,n_unique,pct_unique,sample_values,reasons
active_member,int64,2,0.0002,"[1, 0]","numérico baixa cardinalidade, binário (yes/no)"
credit_card,int64,2,0.0002,"[1, 0]","numérico baixa cardinalidade, binário (yes/no)"


> Esta próxima etapa executa a padronização categórica declarada explicitamente, derivada do diagnóstico da Seção 3.
> As transformações são irreversíveis no pipeline, restritas às features do contrato e sem aplicação de encoding.

In [7]:
from src.features.categorical_standardization import run_categorical_standardization
from src.reporting.ui import render_categorical_standardization_report

# Sem regras categóricas para este dataset
S3_CAT_STD_MAP = {}
S3_CAT_STD_COLS = []

payload_s32 = run_categorical_standardization(
    df=df,    
    scope=scope,
    phrase_map=S3_CAT_STD_MAP,
    cols_scope=S3_CAT_STD_COLS,
)

render_categorical_standardization_report(payload_s32)
df = payload_s32["df"]


Métrica,Antes,Depois,Δ
Linhas,10000.0,10000.0,0.0
Colunas,7.0,7.0,0.0
Memória (MB),0.53,0.53,0.0


> Esta próxima etapa realiza uma auditoria **somente diagnóstica** da coluna target, verificando
> presença, domínio e consistência básica dos valores, **sem aplicar qualquer transformação**.
> O objetivo é validar se o target está adequado para a etapa de modelagem.

In [8]:
from src.features.target_audit import run_target_audit
from src.reporting.ui import render_target_audit_report

# domínio esperado do Telco churn
EXPECTED_TARGET = [0, 1]

payload_s33 = run_target_audit(
    df=df,
    scope=scope,
    expected_values=EXPECTED_TARGET
)

render_target_audit_report(payload_s33)

value,count,pct
0,7963,79.63
1,2037,20.37


## 4 - Tratamento de Dados Faltantes
> Esta seção executa o tratamento explícito de dados faltantes, por meio de imputação controlada e auditável.
> A execução é irreversível no pipeline, restrita às features do contrato, com preservação explícita do target.

In [9]:
from src.features.missing_imputation import run_missing_imputation
from src.reporting.ui import render_missing_imputation_report

# decision: decisão explícita (SEM defaults escondidos)
decision_s4 = {
    # se None: assume intenção = todas as features (ainda filtradas pelo scope)
    "include_cols": None,
    # opcional: colunas explicitamente excluídas
    "exclude_cols": [],
    # obrigatório
    "numeric_strategy": "median",            # "median" | "mean" | "constant"
    "categorical_strategy": "most_frequent", # "most_frequent" | "constant"
}

payload_s4 = run_missing_imputation(df=df, scope=scope, decision=decision_s4)
df = payload_s4["df"]  
render_missing_imputation_report(payload_s4)

Métrica,Antes,Depois,Δ
Linhas,10000.00,10000.00,0.0
Colunas,7.00,7.00,0.0
Memória (MB),0.53,0.53,0.0
column,kind,strategy,fill_value_used
tenure,numeric,median,5.000
estimated_salary,numeric,median,100193.915
active_member,numeric,median,1.000
credit_card,numeric,median,1.000
products_number,numeric,median,1.000
credit_score,numeric,median,652.000


## 5 - Preparação para Modelagem
> Esta seção prepara o dataset para a etapa de modelagem supervisionada, realizando a separação explícita entre variáveis independentes (X) e variável alvo (y), bem como o split entre conjuntos de treino e teste.
> São aplicadas apenas operações estruturais (particionamento e validações), sem transformação de valores, encoding ou engenharia de features.
> A execução inclui diagnósticos auditáveis de integridade, distribuição do target e consistência do escopo, garantindo que os dados estejam aptos para o treinamento dos modelos.

In [10]:
from src.features.contract_and_candidates import NormalizationScope
from src.features.train_test_split_prep import run_train_test_split
from src.reporting.ui import render_modeling_report

decision_s5 = {
    "test_size": 0.2,
    "random_state": 42,
    "shuffle": True,
    "stratify": False,
    # "stratify_col": "Churn",  # só se stratify=True
    "audit_categorical_cardinality": False,  
}

payload_s5 = run_train_test_split(
    df=df,
    scope=scope,
    decision=decision_s5,
)

render_modeling_report(payload_s5)

churn,count_all,rate_all,count_train,rate_train,count_test,rate_test,delta_rate_train_vs_all,delta_rate_test_vs_all
0,7963,0.7963,6356,0.7945,1607,0.8035,-0.0018,0.0072
1,2037,0.2037,1644,0.2055,393,0.1965,0.0018,-0.0072


## 6 - Representação para Modelagem Supervisionada
> Esta seção materializa explicitamente a representação final das variáveis para uso em modelos de Machine Learning supervisionados.
> Aqui são aplicadas as decisões declaradas de representação das features (X) e do target (y), incluindo estratégias de encoding e garantias de consistência entre treino e teste.
> A execução é irreversível no pipeline, restrita ao conjunto de treino para ajuste de transformadores (anti-leakage) e acompanhada de auditorias estruturais que asseguram alinhamento dimensional, rastreabilidade e prontidão para a etapa de modelagem.
> Neste dataset, o target já é fornecido como variável binária (0/1).

In [11]:
from src.features.supervised_representation import run_supervised_representation
from src.reporting.ui import render_supervised_representation_report

# Inputs canônicos 
split_s5 = payload_s5["split"]     # contém: X_train, X_test, y_train, y_test
scope_s5 = payload_s5["scope"]

# Decisão explícita de representação (NADA é inferido)
decision_s6 = {
    "X": {
        "categorical": {
            "strategy": "onehot",
            "handle_unknown": "ignore",
        },
        "numeric": {
            "strategy": "passthrough",
        },
    },
    "y": {
        "strategy": "map_binary",
        "mapping": {0: 0, 1: 1},  # Bank churn já é binário numérico
        "dtype": "int64",
    },
}

payload_s6 = run_supervised_representation(
    split=split_s5,
    scope=scope_s5,
    decision=decision_s6,
)

render_supervised_representation_report(payload_s6)


raw,encoded
0,0
1,1


## 7 - Estratégia de Avaliação e Baselines

> Nesta seção são definidos, de forma explícita, os **critérios de avaliação** do problema de churn.  
> Antes de testar modelos reais, o pipeline estabelece a **métrica principal** e **baselines mínimos**
> para garantir comparações justas e decisões auditáveis nas etapas seguintes.

### Decisão explícita de avaliação

Nesta etapa, o pipeline **não busca otimizar modelos**, mas sim **formalizar o critério de sucesso** do problema de churn.  
As escolhas abaixo representam **decisões normativas**, orientadas por risco de negócio e custo de erro — e não por desempenho empírico observado.

- **`positive_label = 1`**  
  Define explicitamente que a classe positiva representa **churn**.  
  Isso remove ambiguidades semânticas e garante consistência no cálculo das métricas.

- **`primary_metric = "recall"`**  
  Define o **critério de risco mínimo aceitável** do projeto.  
  No contexto de churn, falsos negativos (não identificar um cliente que irá sair) representam perda direta de receita e são considerados o erro mais crítico.
  Essa escolha **não determina automaticamente o modelo final**, nem obriga a ordenação por recall,  
  mas estabelece que **qualquer solução com baixa capacidade de detecção de churn é semanticamente inválida**,  
  independentemente de outras métricas como acurácia ou precisão.


- **`secondary_metrics = ["f1"]`**  
  Métricas de apoio são mantidas para análise complementar, permitindo avaliar o equilíbrio entre precisão e cobertura sem substituir o critério principal.

- **`baselines`**  
  Os modelos baseline (`most_frequent`, `stratified`) servem como **controle mínimo de desempenho**, garantindo que qualquer modelo treinado posteriormente supere estratégias triviais.  
  Eles não competem com os modelos finais — atuam como referência de sanidade.

- **`average = "binary"`**  
  Define explicitamente o escopo de cálculo das métricas, coerente com um problema de classificação binária.

- **`zero_division = 0`**  
  Decisão explícita para evitar comportamento implícito em casos de divisão por zero, garantindo reprodutibilidade e auditabilidade.

- **`random_state = 42`**  
  Assegura que baselines e avaliações sejam **determinísticas**, permitindo comparação justa entre execuções.

> 📌 **Importante:**  
> Esta etapa **define o critério oficial de avaliação**, mas **não impõe** esse critério às explorações da próxima seção.  
> O painel de modelos pode ordenar e comparar resultados por qualquer métrica, mantendo a separação entre **governança avaliativa** e **análise exploratória**.


In [12]:
from src.models.evaluation_baselines import run_section7_evaluation_and_baselines
from src.reporting.ui import render_evaluation_report

# ------------------------------------------------------------
# Decisão explícita 
# ------------------------------------------------------------
decision_s7 = {
    "positive_label": 1,                 # churn = 1
    "primary_metric": "recall",           # métrica principal
    "secondary_metrics": ["f1"],          # métrica de apoio
    "baselines": [
        {"name": "most_frequent", "strategy": "most_frequent"},
        {"name": "stratified", "strategy": "stratified"},
    ],
    "average": "binary",                  # escopo restrito da S7
    "zero_division": 0,                   # decisão explícita
    "random_state": 42,                   # reprodutibilidade
}


# ------------------------------------------------------------
# Execução
# ------------------------------------------------------------
payload_s7 = run_section7_evaluation_and_baselines(
    s6_payload=payload_s6,    # saída auditada da Seção 6
    decision=decision_s7,
)

# ------------------------------------------------------------
# Renderização / Auditoria (UI)
# ------------------------------------------------------------
render_evaluation_report(payload_s7)


## 8 - Painel de Modelos & Hiperparâmetros

Nesta seção, são realizados os **experimentos de Machine Learning** por meio de um **painel interativo**,
permitindo selecionar modelos, configurar hiperparâmetros e definir o modo de treinamento.

O usuário pode:
- selecionar os **modelos** a serem avaliados,
- escolher entre **treino direto** ou **busca de hiperparâmetros (GridSearchCV)**,
- configurar parâmetros via **painel** ou **dict**,
- executar os experimentos **explicitamente** pelo botão **Rodar experimento**.

> ⚠️ O experimento **só é executado** ao clicar em **Rodar experimento**.

### Avaliação e métricas

- As **métricas são calculadas nesta etapa, utilizando o conjunto de teste.
- A etapa anterior, Estratégia de Avaliação e Baselines, pode sugerir um critério principal de decisão (ex.: *recall*),
  mas o **Leaderboard pode ser ordenado por qualquer métrica**, sem restrições.

### Grids de hiperparâmetros

Os grids utilizados na busca de hiperparâmetros seguem escolhas **deliberadas e documentadas**,
com foco em equilíbrio entre rigor acadêmico, custo computacional e reprodutibilidade.

A fundamentação completa está disponível no documento oficial:

📄 **`hyperparameter_grids.md`** — referência única do projeto para decisões sobre grids.


In [13]:
from src.reporting.models_control_panel import render_section8_models_panel

render_section8_models_panel(payload_s6=payload_s6, payload_s7=payload_s7, payload_s5=payload_s5)


### ✅ Decisão da escolha do modelo (pós-processamento)

Com base nos experimentos realizados nesta seção, a decisão do modelo considera o **critério de avaliação definido na Seção 7** (Estratégia de Avaliação e Baselines) e os **trade-offs observados no Leaderboard**.

A **Decision Tree** apresentou o **maior recall**, alinhando-se diretamente ao critério de risco do problema de churn, cujo objetivo é **maximizar a detecção de clientes propensos a sair**, mesmo ao custo de mais falsos positivos.  
Por outro lado, o **Random Forest** obteve o **melhor F1-score** e o maior **ROC-AUC**, representando a opção com **melhor equilíbrio geral** entre precisão e recall, além de maior capacidade discriminativa.

> **Síntese inicial:**  
> - **Escolha orientada por risco (S7 — recall):** *Decision Tree*  
> - **Escolha orientada por desempenho equilibrado (S8 — F1):** *Random Forest*

Apesar da Decision Tree estar mais diretamente alinhada ao critério de risco definido na Seção 7, a escolha do modelo a ser **exportado** considera também aspectos de **robustez, estabilidade estatística e generalização**.

O **Random Forest** foi selecionado para exportação porque:
- apresentou o **melhor F1-score**, indicando maior equilíbrio entre *recall* e *precision*;
- obteve o maior **ROC-AUC**, demonstrando **melhor capacidade de separação** entre clientes churn e não churn;
- reduz significativamente o risco de **overfitting**, comum em árvores de decisão individuais;
- oferece maior **estabilidade**, sendo menos sensível a variações do conjunto de treino.

Do ponto de vista operacional, essa escolha **não invalida o critério de risco estabelecido na Seção 7**.  
Ao contrário, ela preserva a possibilidade de **ajuste posterior do limiar de decisão (threshold)**, permitindo elevar o *recall* conforme a necessidade do negócio, sem comprometer a capacidade de generalização do modelo.

> 📌 **Síntese final da decisão:**  
> - A **Seção 7** define **qual tipo de erro é inaceitável** (governança de risco).  
> - A **Seção 8** seleciona o modelo com **maior capacidade discriminativa global** para atender a esse critério de forma robusta.  
> - O **Random Forest** representa o melhor ponto de partida para **uso operacional e eventual calibração**.

---

📐 **Nota metodológica sobre os grids de hiperparâmetros**

Os resultados apresentados nesta seção foram obtidos a partir de **grids de hiperparâmetros deliberadamente controlados**, projetados para garantir:
- comparabilidade justa entre modelos,
- custo computacional previsível,
- e reprodutibilidade dos experimentos.

A definição e a fundamentação acadêmica desses grids estão documentadas no arquivo  
**`hyperparameter_grids.md`**, que serve como **referência oficial do projeto** para decisões relacionadas à busca de hiperparâmetros.

Essa escolha metodológica assegura que a decisão do modelo não seja resultado de *tuning excessivo*,  
mas da **capacidade discriminativa intrínseca** dos algoritmos avaliados.


## 9 - Exportação do Modelo

Nesta seção, o modelo selecionado na Seção 8 é materializado como um artefato persistente,
pronto para uso em inferência futura.

Nenhum novo treinamento ou avaliação é realizado.
O objetivo é garantir que o modelo exportado preserve:
- a representação de dados definida na Seção 6;
- os parâmetros aprendidos durante o treinamento;
- a decisão empírica documentada na Seção 8.


In [15]:
# ============================================================
# Seção 9 — Exportação do Modelo Treinado (Caminho A)
# ============================================================

from src.reporting.models_control_panel import get_payload_s8
from sklearn.pipeline import Pipeline
from pathlib import Path
import joblib

# ------------------------------------------------------------
# 1) Recupera artefatos produzidos pela S8
# ------------------------------------------------------------
payload_s8 = get_payload_s8()

# ------------------------------------------------------------
# 2) Escolha explícita do modelo a exportar
#    (nenhuma decisão automática)
# ------------------------------------------------------------
MODEL_KEY = "rf"  # ex.: "rf", "dt", "knn", "logreg"
best_estimator = payload_s8["best_estimators"][MODEL_KEY]

# ------------------------------------------------------------
# 3) Monta pipeline final (S6 + estimador treinado)
# ------------------------------------------------------------
final_pipeline = Pipeline(steps=[
    ("preprocess", payload_s6["representation"]["transformer"]),
    ("model", best_estimator),
])

# ------------------------------------------------------------
# 4) Resolve path correto para artifacts/ na raiz do projeto
# ------------------------------------------------------------
project_root = Path("..").resolve()   # notebooks/ → raiz
artifacts_dir = project_root / "artifacts"
artifacts_dir.mkdir(exist_ok=True)

out_path = artifacts_dir / "churn_model.joblib"

# ------------------------------------------------------------
# 5) Exporta artefato treinado
# ------------------------------------------------------------
joblib.dump(final_pipeline, out_path)

print(f"[INFO] Modelo exportado com sucesso em: {out_path}")


[INFO] Modelo exportado com sucesso em: C:\Users\fabio\Projetos DEV\Hackathon ONE\churninsight-hackathon-one\artifacts\churn_model.joblib
